# CS436 PA1 Image Warping

# Importing Libraries

This might take a few seconds. If any library is missing, make sure to install it in your environment, using anaconda in for library installation is usually easier

In [ ]:
import matplotlib
import imageio
from mpl_toolkits import mplot3d
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib import image as mpimg
from matplotlib.pyplot import figure
%matplotlib inline

import seaborn as sns
import numpy as np
import matplotlib.pylab as pl
from matplotlib.colors import ListedColormap

# Helper Functions 

Interpolation function for Task 1 

In [ ]:
def InterpolationNearest(img, TransformedWithoutInterpolation, xBound, yBound, row, col):
    
    TransformedWithInterpolation = np.zeros((xBound, yBound, 3)).astype(int)
    Black = np.array(
                [0,0,0]
            )
    # finding the nearest neighbour and mapping it      
    for x in range(xBound):
        for y in range(yBound):
            present = np.array(
                        [x, y, 1]
                    )
            value = np.matmul(TransformedWithoutInterpolation, present)
            value = np.round(value)
            Foundx = int(value[0])
            Foundy = int(value[1])
            
            # checking if in bound or not
            if Foundx < 0 or Foundx >= row:
                TransformedWithInterpolation[x][y] = Black
            elif Foundy < 0 or Foundy >= col:
                TransformedWithInterpolation[x][y] = Black
            else:
                TransformedWithInterpolation[x][y] = img[Foundx, Foundy]
    
    return TransformedWithInterpolation

# 1. Image Warping

In this section, <b>you are not allowed to use any OpenCV functions.</b> OpenCV comes in the next section

## Task 1.1 - Part A (Projective Transformations)


In this section you will be creating a function `warpPerspective(img,M,(cols,rows))`. 
The function performs a projective transformation on an image and returns a new image variable
    
    This function has the following arguments:
        img: original image variable. This can be a black and white or a color image
        M: a 3x3 projective transformation matrix
        cols: the columns of the output image
        rows: the rows of the output matrix        

The function can transform a small image to a larger image. While doing so, the new image must not have any fringes or emply pixels, you can fill those pixels by taking the value of the nearest neigbouring pixel. The pixels outside of an image may be set to black for now. You may change the arguments of the function if you want to add additional features

In [ ]:
def warpPerspective(img, M, dsize):
    dst = M.copy()
    col = dsize[0]
    row = dsize[1]
    
    # The corner points are (0,0) (0, col) (row,0) (row, col) so
    OriginalCornerPoints = np.array(
        [
            [0, row, 0, row],
            [0, 0, col, col],
            [1, 1, 1, 1]
        ]
    )
    
    # The transformation matrix M is being multiplied with the original corner points
    TransformedCornerPoints = np.matmul(M, OriginalCornerPoints)
    
    minimum = np.amin(TransformedCornerPoints, axis = 1)
    Xminimum = minimum[0]
    Yminimum = minimum[1]
    
    maximum = np.amax(TransformedCornerPoints, axis = 1)
    Xmaximum = maximum[0]
    Ymaximum = maximum[1]
    
    xBound = Xmaximum - Xminimum
    xBound = np.round(xBound)
    xBound = int(xBound)
    
    yBound = Ymaximum - Yminimum
    yBound = np.round(yBound)
    yBound = int(yBound)
    
    TranslationValues = np.amin(TransformedCornerPoints, axis=1)
    Tx = TranslationValues[0]
    Ty = TranslationValues[1]
    
    TranslationMatrix = np.array(
        [
            [1, 0, -Tx],
            [0, 1, -Ty],
            [0, 0, 1]
        ]
    )
    
    TransformedWithoutInterpolation = np.matmul(np.linalg.inv(M), np.linalg.inv(TranslationMatrix))
    
    TransformedWithInterpolation = InterpolationNearest(img, TransformedWithoutInterpolation, xBound, yBound, row, col)
    
    return TransformedWithInterpolation

## Task 1.1 - Part B: Rotation



In this test case, we will be rotating a subsection of an image of a cat. We will convert `cat.jpg` to `cat_sol0_1.jpg`



We can utilize your newly made function for that with the following steps

1. Read the file `part1/cat.jpg`
2. Crop it to the portion that shows just the head
3. Input it to your function with your projective matrix
4. Display the output
5. You can adjust the row and col values such that the entirety of the output image is shown
    
The output should look similar to `part1/cat_sol0_1.jpg`. You can try coming up with a transformation on paper before using it. Affine transformations are a subset of projective transformations, so you can try starting from those

# My Solution

1. Reading the file part1/cat.jpg

In [ ]:
cat = np.array(mpimg.imread('part1/cat.jpg'))
cat.setflags(write=1)

2. Cropping it to the portion that shows just the head

-> For that we need to first determine what the range of x and y values we take into consideration

In [ ]:
plt.imshow(cat)

This gives us a rough idea of what the dimensions should be. Lets consider 5 to 155 for the y axis and 82 to 200 for the x axis.

In [ ]:
Cropped = cat[5:155, 82:200]
plt.imshow(Cropped)

3. Input it to the warpPerspective with a projective matrix

-> First lets gather the appropriate parameters

In [ ]:
ShapeDimension = Cropped.shape
row = ShapeDimension[0] - 1
col = ShapeDimension[1] - 1
size = (col, row)

angleRad = np.radians(30)
RotationMatrix = np.array(
                            [
                                [np.cos(angleRad), -np.sin(angleRad), 0],
                                [np.sin(angleRad), np.cos(angleRad), 0],
                                [0, 0, 1]
                            ]
                        )

Transformed = warpPerspective(Cropped, RotationMatrix, size)

4. Display the output

In [ ]:
plt.imshow(Transformed)

# For comparison, this is the sample output given to us:

In [ ]:
catAns = np.array(mpimg.imread('part1/cat_sol0_1.jpg'))
catAns.setflags(write=1)
plt.imshow(catAns)

# 2. Interpolation

At this point, you can predict that creating a large image from small image will not give us the best results, ie there will be a lot of empty pixel values. There are multiple ways to handle this, i.e. Bilinear Interpolation, Nearest, Bicubic etc. However, the method that we will be utilizing in this assignment is fairly simple. We will be replacing empty pixel values with the average of all pixel values around that pixel. Be careful, while handling the corner/side pixels of the image.



# Modifying warpPerspective

In [ ]:
def Interpolation(img, TransformedWithoutInterpolation, row, col, xBound, yBound):
    
    TransformedWithInterpolation = np.zeros((row, col, 3)).astype(int)
    
    # finding the nearest neighbour and mapping it      
    for x in range(row):
        for y in range(col):
            present = np.array(
                        [x, y, 1]
                    )
            
            value = np.matmul(TransformedWithoutInterpolation, present)
            value = np.round(value)
            Foundx = int(value[0])
            Foundy = int(value[1])
            # checking if in bound or not
            if Foundx < 0 or Foundx >= xBound:
                continue
            elif Foundy < 0 or Foundy >= yBound:
                continue
            elif Foundx == 0 and Foundy == 0:
                arg1 = img[Foundx + 1, Foundy]
                arg2 = img[Foundx, Foundy + 1]
                arg3 = img[Foundx + 1, Foundy + 1]
                
                x1 = int(arg1[0])
                x2 = int(arg2[0])
                x3 = int(arg3[0])
                averageX = int((x1 + x2 + x3) / 3)
                
                y1 = int(arg1[1])
                y2 = int(arg2[1])
                y3 = int(arg3[1])
                averageY = int((y1 + y2 + y3) / 3)
                
                z1 = int(arg1[2])
                z2 = int(arg2[2])
                z3 = int(arg3[2])
                averageZ = int((z1 + z2 + z3) / 3)
                
                
                Transformed = np.array(
                                        [averageX, averageY, averageZ]
                                )
                TransformedWithInterpolation[x][y] = Transformed
            elif Foundx == 0 and Foundy == yBound-1:
                arg1 = img[Foundx + 1, Foundy]
                arg2 = img[Foundx, Foundy - 1]
                arg3 = img[Foundx + 1, Foundy - 1]
                
                x1 = int(arg1[0])
                x2 = int(arg2[0])
                x3 = int(arg3[0])
                averageX = int((x1 + x2 + x3) / 3)
                
                y1 = int(arg1[1])
                y2 = int(arg2[1])
                y3 = int(arg3[1])
                averageY = int((y1 + y2 + y3) / 3)
                
                z1 = int(arg1[2])
                z2 = int(arg2[2])
                z3 = int(arg3[2])
                averageZ = int((z1 + z2 + z3) / 3)
                
                
                Transformed = np.array(
                                        [averageX, averageY, averageZ]
                                )
                TransformedWithInterpolation[x][y] = Transformed
            elif Foundx == xBound-1 and Foundy == yBound-1:
                arg1 = img[Foundx - 1, Foundy]
                arg2 = img[Foundx, Foundy - 1]
                arg3 = img[Foundx - 1, Foundy - 1]
                
                x1 = int(arg1[0])
                x2 = int(arg2[0])
                x3 = int(arg3[0])
                averageX = int((x1 + x2 + x3) / 3)
                
                y1 = int(arg1[1])
                y2 = int(arg2[1])
                y3 = int(arg3[1])
                averageY = int((y1 + y2 + y3) / 3)
                
                z1 = int(arg1[2])
                z2 = int(arg2[2])
                z3 = int(arg3[2])
                averageZ = int((z1 + z2 + z3) / 3)
                
                
                Transformed = np.array(
                                        [averageX, averageY, averageZ]
                                )
                TransformedWithInterpolation[x][y] = Transformed
            elif Foundx == xBound-1 and Foundy == 0:
                arg1 = img[Foundx - 1, Foundy]
                arg2 = img[Foundx, Foundy + 1]
                arg3 = img[Foundx - 1, Foundy + 1]
                
                x1 = int(arg1[0])
                x2 = int(arg2[0])
                x3 = int(arg3[0])
                averageX = int((x1 + x2 + x3) / 3)
                
                y1 = int(arg1[1])
                y2 = int(arg2[1])
                y3 = int(arg3[1])
                averageY = int((y1 + y2 + y3) / 3)
                
                z1 = int(arg1[2])
                z2 = int(arg2[2])
                z3 = int(arg3[2])
                averageZ = int((z1 + z2 + z3) / 3)
                
                
                Transformed = np.array(
                                        [averageX, averageY, averageZ]
                                )
                TransformedWithInterpolation[x][y] = Transformed
            elif Foundx == 0:
                arg1 = img[Foundx + 1, Foundy]
                arg2 = img[Foundx, Foundy + 1]
                arg3 = img[Foundx, Foundy - 1]
                
                x1 = int(arg1[0])
                x2 = int(arg2[0])
                x3 = int(arg3[0])
                averageX = int((x1 + x2 + x3) / 3)
                
                y1 = int(arg1[1])
                y2 = int(arg2[1])
                y3 = int(arg3[1])
                averageY = int((y1 + y2 + y3) / 3)
                
                z1 = int(arg1[2])
                z2 = int(arg2[2])
                z3 = int(arg3[2])
                averageZ = int((z1 + z2 + z3) / 3)
                
                
                Transformed = np.array(
                                        [averageX, averageY, averageZ]
                                )
                TransformedWithInterpolation[x][y] = Transformed
            elif Foundy == 0:
                arg1 = img[Foundx - 1, Foundy]
                arg2 = img[Foundx + 1, Foundy]
                arg3 = img[Foundx, Foundy + 1]
                
                x1 = int(arg1[0])
                x2 = int(arg2[0])
                x3 = int(arg3[0])
                averageX = int((x1 + x2 + x3) / 3)
                
                y1 = int(arg1[1])
                y2 = int(arg2[1])
                y3 = int(arg3[1])
                averageY = int((y1 + y2 + y3) / 3)
                
                z1 = int(arg1[2])
                z2 = int(arg2[2])
                z3 = int(arg3[2])
                averageZ = int((z1 + z2 + z3) / 3)
                
                
                Transformed = np.array(
                                        [averageX, averageY, averageZ]
                                )
                TransformedWithInterpolation[x][y] = Transformed
            elif Foundx == xBound - 1:
                arg1 = img[Foundx - 1, Foundy]
                arg2 = img[Foundx, Foundy - 1]
                arg3 = img[Foundx, Foundy + 1]
                
                x1 = int(arg1[0])
                x2 = int(arg2[0])
                x3 = int(arg3[0])
                averageX = int((x1 + x2 + x3) / 3)
                
                y1 = int(arg1[1])
                y2 = int(arg2[1])
                y3 = int(arg3[1])
                averageY = int((y1 + y2 + y3) / 3)
                
                z1 = int(arg1[2])
                z2 = int(arg2[2])
                z3 = int(arg3[2])
                averageZ = int((z1 + z2 + z3) / 3)
                Transformed = np.array(
                                        [averageX, averageY, averageZ]
                                )
                TransformedWithInterpolation[x][y] = Transformed
            elif Foundy == yBound - 1:
                arg1 = img[Foundx - 1, Foundy]
                arg2 = img[Foundx + 1, Foundy]
                arg3 = img[Foundx, Foundy - 1]
                
                x1 = int(arg1[0])
                x2 = int(arg2[0])
                x3 = int(arg3[0])
                averageX = int((x1 + x2 + x3) / 3)
                
                y1 = int(arg1[1])
                y2 = int(arg2[1])
                y3 = int(arg3[1])
                averageY = int((y1 + y2 + y3) / 3)
                
                z1 = int(arg1[2])
                z2 = int(arg2[2])
                z3 = int(arg3[2])
                averageZ = int((z1 + z2 + z3) / 3)
                
                Transformed = np.array(
                                        [averageX, averageY, averageZ]
                                )
                TransformedWithInterpolation[x][y] = Transformed
            else:
                arg1 = img[Foundx + 1, Foundy]
                arg2 = img[Foundx, Foundy - 1]
                arg3 = img[Foundx + 1, Foundy - 1]
                arg4 = img[Foundx, Foundy]
                
                x1 = int(arg1[0])
                x2 = int(arg2[0])
                x3 = int(arg3[0])
                x4 = int(arg4[0])
                averageX = int((x1 + x2 + x3 + x4) / 4)
                
                y1 = int(arg1[1])
                y2 = int(arg2[1])
                y3 = int(arg3[1])
                y4 = int(arg4[1])
                averageY = int((y1 + y2 + y3 + y4) / 4)
                
                z1 = int(arg1[2])
                z2 = int(arg2[2])
                z3 = int(arg3[2])
                z4 = int(arg4[2])
                averageZ = int((z1 + z2 + z3 + z4) / 4)
                
                Transformed = np.array(
                                        [averageX, averageY, averageZ]
                                )
                TransformedWithInterpolation[x][y] = Transformed
    return TransformedWithInterpolation

In [ ]:
def warpPerspective(img, M, dsize, newSize):
    dst = M.copy()
    col = dsize[0]
    row = dsize[1]
    
    # The corner points are (0,0) (0, col) (row,0) (row, col) so
    OriginalCornerPoints = np.array(
        [
            [0, row, 0, row],
            [0, 0, col, col],
            [1, 1, 1, 1]
        ]
    )
    
    TransformedCornerPoints = np.matmul(M, OriginalCornerPoints)
    minimum = np.amin(TransformedCornerPoints, axis = 1)
    Xminimum = minimum[0]
    Yminimum = minimum[1]
    
    maximum = np.amax(TransformedCornerPoints, axis = 1)
    Xmaximum = maximum[0]
    Ymaximum = maximum[1]
    
    xBound = Xmaximum - Xminimum
    xBound = np.round(xBound)
    xBound = int(xBound)
    
    yBound = Ymaximum - Yminimum
    yBound = np.round(yBound)
    yBound = int(yBound)
    
    TranslationValues = np.amin(TransformedCornerPoints, axis=1)
    Tx = TranslationValues[0]
    Ty = TranslationValues[1]
    
    TranslationMatrix = np.array(
        [
            [1, 0, -Tx],
            [0, 1, -Ty],
            [0, 0, 1]
        ]
    )
    
    TransformedWithoutInterpolation = np.matmul(np.linalg.inv(M), np.linalg.inv(TranslationMatrix))
    
    return TransformedWithoutInterpolation

## Task 2.1


For this, we can augment your function `warpPerspective(img,M,(cols,rows))`.

1. Where you previously took the nearest neighbor to fill missing pixels, use the `Interpolation` algorithm. 
2. Input cols and rows values as 2000 and 2160
3. Use the same projective matrix
4. Display the output
    
the output should look similar to `part1/cat_sol1.jpg`.

In [ ]:
cols = 2000
rows = 2160
size = (cols, rows)
row = ShapeDimension[0] - 1
col = ShapeDimension[1] - 1
orgSize = (col, row)
angleRad = np.radians(30)
RotationMatrix = np.array(
                            [
                                [np.cos(angleRad), -np.sin(angleRad), 0],
                                [np.sin(angleRad), np.cos(angleRad), 0],
                                [0, 0, 1]
                            ]
                        )

scaling = np.array(
            [
                [11.2, 0, 0],
                [0, 11.2, 0],
                [0, 0, 1]
            ]
        )

M = np.matmul(RotationMatrix, scaling)
TransformedWithoutInterpolation = warpPerspective(Cropped, M, orgSize ,size)

TransformedWithInterpolation = Interpolation(Cropped, TransformedWithoutInterpolation, rows, cols, row, col)

In [ ]:
TransformedWithInterpolation.setflags(write=1)
plt.imshow(TransformedWithInterpolation)

# For comparison, this is the sample output given to us:

In [ ]:
catAns2 = np.array(mpimg.imread('part1/cat_sol1.jpg'))
catAns2.setflags(write=1)
plt.imshow(catAns2)

# 3. Image Alpha Values



In this section we will be blending two images to create a third image. This process is important in applications such as VR, AR etc.

For this section, it is important to understand alpha values. Alpha values range between 0.0 and 1.0, and determine the transparency of a pixel. These are common in png files where they are stored alongside rgb as rgba. 

## Task 3.1

You must manipulate your `warpPerspective(img,M,(cols,rows))` function so that it returns values in the form of rgba. You can do this with the following steps: 

1. If the image has values in 8bit integers, convert your 0 - 255 color integer value range to 0.0 - 1.0 float. 
2. Append a forth pixel value for the alpha
3. It might be easier to create a seperate function that converts rgb to rgba with alpha values set to 1.0
4. Where you were previously setting the background to be black, also set the alpha to 0.0
    
The output from the previous test should look similar to `part1/cat_sol2.png`. you can explore the png file structure to get a better idea of the output. 




In [ ]:
def warpPerspective3(img, M, dsize):
    dst = M.copy()
    
    dimensions = img.shape
    row = dimensions[0] - 1
    col = dimensions[1] - 1
    
    OriginalCornerPoints = np.array(
        [
            [0, row, 0, row],
            [0, 0, col, col],
            [1, 1, 1, 1]
        ]
    )
    
    TransformedCornerPoints = np.matmul(M, OriginalCornerPoints)
    minimum = np.amin(TransformedCornerPoints, axis = 1)
    Xminimum = minimum[0]
    Yminimum = minimum[1]
    
    maximum = np.amax(TransformedCornerPoints, axis = 1)
    Xmaximum = maximum[0]
    Ymaximum = maximum[1]
    
    xBound = Xmaximum - Xminimum
    xBound = np.round(xBound)
    xBound = int(xBound)
    
    yBound = Ymaximum - Yminimum
    yBound = np.round(yBound)
    yBound = int(yBound)
    
    TranslationValues = np.amin(TransformedCornerPoints, axis=1)
    Tx = TranslationValues[0]
    Ty = TranslationValues[1]
    
    TranslationMatrix = np.array(
        [
            [1, 0, -Tx],
            [0, 1, -Ty],
            [0, 0, 1]
        ]
    )
    
    TransformedWithoutInterpolation = np.matmul(np.linalg.inv(M), np.linalg.inv(TranslationMatrix))
    
    newCol = dsize[0]
    newRow = dsize[1]
    
    TransformedWithInterpolation = np.zeros((rows, cols, 4))
    
    for x in range(newRow):
        for y in range(newCol):
            present = np.array(
                        [x, y, 1]
                    )
            value = np.matmul(TransformedWithoutInterpolation, present)
            
            Foundx = int(value[0])
            Foundy = int(value[1])
            
            if Foundx < 0 or Foundx >= row:
                continue
            elif Foundy < 0 or Foundy >= col:
                continue
            else:
                Top = Sides = img[Foundx, Foundy]
                Bottom = img[Foundx + 1, Foundy]
                conversion = 255.0
                arg1 = Sides[0] / conversion
                arg2 = Sides[1] / conversion
                arg3 = Sides[2] / conversion
                
                transformed = np.array(
                                    [arg1, arg2, arg3, 1.0]
                                ) 
                TransformedWithInterpolation[x][y] = transformed
    return TransformedWithInterpolation

In [ ]:
scalingFactor = 10
s = np.array(
    [
        [scalingFactor, 0, 0],
        [0, scalingFactor, 0],
        [0, 0, 1]
    ]
    )

Matrix = np.matmul(RotationMatrix, s)
cols = 2000
rows = 2160

output = warpPerspective3(Cropped, Matrix, (cols, rows))

In [ ]:
plt.imshow(output)

# 4. Cricket Stadium

In this part we will be adding an image of a cricket scoreboard to a cricket stadium.



In [ ]:
Ans4 = np.array(mpimg.imread('stadium-sol5.jpg'))
Ans4.setflags(write=1)
plt.imshow(Ans4)

## Task 4.1

For this task, you need to transform the given scoreboard image. Your output should look similar to `transformed-scoreboard.jpg`. You can try coming up with a transformation on paper before using it. Affine transformations are a subset of projective transformations, so you can try starting from those.

In [ ]:
Ans41 = np.array(mpimg.imread('transformed-scoreboard.jpg'))
Ans41.setflags(write=1)
plt.imshow(Ans41)

In [ ]:
# start your code here
scoreBoard = np.array(mpimg.imread('score.jpg'))
scoreBoard.setflags(write=1)
plt.imshow(scoreBoard)

In [ ]:
rowI = 300
colI = 640

In [ ]:
dimensions = scoreBoard.shape
width = dimensions[1]
height = dimensions[0]

In [ ]:
plt.imshow(Ans41)

In [ ]:
import cv2

OriginalCornerPoints = np.float32(
        [
            [0, 0],
            [width-1,0],
            [width-1, height-1],
            [0,height-1]
        ]   
        )

dst = np.float32([
    [395, 220],
    [550, 220],
    [570, 290],
    [370, 290]]
    )

M = cv2.getPerspectiveTransform(OriginalCornerPoints, dst)
w = cv2.warpPerspective(scoreBoard, M, (colI, rowI), borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))    
plt.imshow(w)

## Task 4.2

After you have transformed the scorebaord you have to add it to the given stadium image. This can be done by copying pixel values from the transformed scoreboard image to the given stadium image. You need to figure out the appropriate conditions for copying the pixel values. 

In [ ]:
stadium = np.array(mpimg.imread('PSL-Stadium.jpg'))
stadium.setflags(write=1)
plt.imshow(stadium)

In [ ]:
for x in range(rowI):
    for y in range(colI):
        if w[x][y][0] < 90 and w[x][y][1] < 90 and w[x][y][2] < 90:
            stadium[x][y] = w[x][y]
plt.imshow(stadium)

In [ ]:
plt.imshow(Ans4)

## Task 4.3

In certain situations, when we want to isolate a subject in an image, alpha masks are provided to us. Alpha masks describe the transparency of an image. These masks can be in any format ranging from jpg, png, bmp, gif,etc in our example we will be using jpg.

For this part we will reproduce the same solution but by using alpha mask this time. Below you are given an alpha mask for the transfomed scoreboard. Using this alpha mask you need to combine the two images together to produce the same output.

You can achieve the desired solution as follows:

Consider the transformed scoreboard image as the foreground image and the stadium image as the background image. At every pixel of the image, we need to combine the foreground image pixel value with the background image pixel value using the alpha mask.

 
\begin{align*}
T = \alpha*F+(1-\alpha)B
\end{align*}

1. Where T is target pixel value
2. F is the pixel value in the foreground image
3. B is the pixel value of the background image
4. Alpha is value of the alpha mask for the corresponding pixels

In [ ]:
# score-mask.jpg
mask = np.array(mpimg.imread('score_mask.jpg'))
mask.setflags(write=1)
plt.imshow(mask)

In [ ]:
stadium = np.array(mpimg.imread('PSL-Stadium.jpg'))
stadium.setflags(write=1)
plt.imshow(stadium)

In [ ]:
scoreBoard = np.array(mpimg.imread('tscoreboard.jpg'))
scoreBoard.setflags(write=1)
plt.imshow(scoreBoard)

In [ ]:
# start your code here
Dimensions = scoreBoard.shape
row = Dimensions[0] 
col = Dimensions[1]
size = (row, col, 4)
array = np.zeros(size)
alphaValue = 1.0
FinalOutput = np.zeros(size)
Tstadium = np.zeros(size)

for x in range(row):
    for y in range(col):
        if mask[x][y][0] < 100 and mask[x][y][1] < 100 and mask[x][y][2] < 100:
            continue
        else:
            arg1 = scoreBoard[x][y][0]/255
            arg2 = scoreBoard[x][y][1]/255
            arg3 = scoreBoard[x][y][2]/255
          
            value = np.array(
                    [arg1, arg2, arg3, alphaValue]
                  )
            array[x][y] = value

for x in range(row):
    for y in range(col):
        arg1 = stadium[x][y][0]/255
        arg2 = stadium[x][y][1]/255
        arg3 = stadium[x][y][2]/255
        value = np.array(
                    [arg1, arg2, arg3, alphaValue]
                  )
        Tstadium[x][y] = value

for x in range(row):
    for y in range(col):
        alpha = array[x][y][3]
        B = Tstadium[x][y]
        T = (alpha * array[x][y]) + ((1 - alpha)* B)
        FinalOutput[x][y] = T
plt.imshow(FinalOutput)

# 5. Solar System (CS5310, EE513 Only, Upto 10 marks bonus for CS436)

![alt Solar System demo](https://drive.google.com/uc?id=10xxjq46hha2HS0xjyaqM-iwgrMg01d6w)

In [ ]:
height = width = 1500 # shape of canvas

## Task 5.1
In this task you will try to recreate the above gif by making use of all the previously implemented techniques.

Your first task is to import the image of planet earth and apply appropriate transformations to center it within the canvas. 

You are allowed to import opencv and make use of cv2.warpPerspective(sun,M,(width, height)) 



In [ ]:
# Import sun.jpg and apply appropriate transformations to center the image

Your result should look like (It does not matter whether the background is black or white)

Note: The resulting canvas has height and width of 1500px

![alt Sun Result](https://drive.google.com/uc?id=1IdYgql7RFwvXT1SxwuBRkW3AMKdZeexg)

## Task 5.2
In this part, you will carry out the same process for earth.jpg as well. Except, in this case the earth needs to be translated +400px on the y axis.

Your final image should look something like this

![alt Result earth](https://drive.google.com/uc?id=16eFaOHVlhUKgHpw8ZVCwSvzagJcD_s8K)

## Task 5.3
You will now modify the above code to apply rotations to the earth as such it rotates along its own axis as well as around the sun. (This is same as the problem given to you in your written assignment 1)

Feel free to copy your code from task 5.2

In [ ]:
earth_func(earth,hour):
  #earth -> this is the imported earth.jpg file in numpy array format
  #hour -> You will use this parameter to caculate the rotation of earth around the sun and along its own axis. For example,
  # if hour == 15 then rotation on its own axis will be 225 deg whilst rotations around the sun will be 0.61 deg
  # return val will be the transformed image of earth such as if hours == 0 the resulting image will be the same as task 5.2
  return earth_

In [ ]:
earths = []
for x in range(30):
  earths.append(earth_func(earth,x*10))

## Task 5.4
You will now blend all three images to compose a single image. Feel free to make use of the canvas below.

You will have to traverse every element of the list (earths) and blend them with the transformed sun image created in task 5.1. After doing this, you will get an array of images in the end with each having a sun and a earth in them. Store this array in the variable final and run the code gif builder code in the end

In [ ]:
# Code for canvas
canvas = np.array([255]*3*width*height)
canvas.resize((width,height,3))
plt.imshow(canvas)

In [ ]:
# gif builder code
# final is a list of numpy images 
imageio.mimsave('movie.gif', final)

Please submit movie.gif along with the jupyter notebook

# Submission Instructions

You are required to submit the CV_PA1 Jupyter file and Part-5 gif only via LMS. Rename CV_PA1 to CV_PA1_[your rollnumber]. 

The submitted file should be in a readable and understandable format. Add as many comments as possible. It should run from start to end without any errors. It must be in Python3.

The notebook "CV_PA1_intro" will help you get started with image manipulation. If you are familiar with it, skip it.
